#### Question 4

## Naive Bayes - Play Tennis Dataset

**Mục tiêu:** Dự đoán có nên chơi tennis hay không dựa trên 4 yếu tố thời tiết:
- Outlook (Thời tiết): Sunny, Overcast, Rain
- Temperature (Nhiệt độ): Hot, Mild, Cool
- Humidity (Độ ẩm): High, Normal
- Wind (Gió): Weak, Strong

**Công thức Naive Bayes:**
```
P(Y|X) = P(Y) × P(X₁|Y) × P(X₂|Y) × ... × P(Xₙ|Y)
```

Trong đó:
- P(Y): Xác suất tiên nghiệm (Prior Probability) - tỷ lệ xuất hiện của lớp Y
- P(Xᵢ|Y): Xác suất có điều kiện (Conditional Probability) - xác suất xuất hiện giá trị Xᵢ khi biết lớp Y

In [23]:
#########################
# Create data
#########################

# 4.1

import numpy as np

def create_train_data():
  # Đọc file bang1.txt với delimiter là dấu phẩy
  data = np.loadtxt("bang1.txt", delimiter=",", dtype=str)
  return data

train_data = create_train_data()
print(train_data)

[['Sunny' 'Hot' 'High' 'Weak' 'no']
 ['Sunny' 'Hot' 'High' 'Strong' 'no']
 ['Overcast' 'Hot' 'High' 'Weak' 'yes']
 ['Rain' 'Mild' 'High' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Cool' 'Normal' 'Strong' 'no']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'yes']
 ['Sunny' 'Mild' 'High' 'Weak' 'no']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'yes']
 ['Rain' 'Mild' 'Normal' 'Weak' 'yes']]


In [24]:
# ============================================
# BƯỚC 2: TÍNH XÁC SUẤT TIÊN NGHIỆM P(Y)
# ============================================

def compute_prior_probablity(train_data):
  """
  Tính xác suất tiên nghiệm P(Y) cho mỗi lớp
  
  Công thức: P(Y=y) = số lượng mẫu có nhãn y / tổng số mẫu
  
  Ví dụ: Nếu có 10 mẫu, 4 mẫu 'no' và 6 mẫu 'yes'
         => P(Y='no') = 4/10 = 0.4
         => P(Y='yes') = 6/10 = 0.6
  
  Args:
    train_data: numpy array dữ liệu huấn luyện
    
  Returns:
    numpy array [P(Y='no'), P(Y='yes')]
  """
  y_unique = ['no', 'yes']
  prior_probability = np.zeros(len(y_unique))
  
  # Đếm số lần xuất hiện của mỗi lớp
  for i in range(len(y_unique)):
    count = np.sum(train_data[:, -1] == y_unique[i])
    prior_probability[i] = count / len(train_data)
  
  return prior_probability

# Test
prior_probability = compute_prior_probablity(train_data)
print(f"P(Play Tennis = No) = {prior_probability[0]}")
print(f"P(Play Tennis = Yes) = {prior_probability[1]}")

P(Play Tennis = No) = 0.4
P(Play Tennis = Yes) = 0.6


In [25]:
# ============================================
# BƯỚC 3: TÍNH XÁC SUẤT CÓ ĐIỀU KIỆN P(X|Y)
# ============================================

def compute_conditional_probability(train_data):
  """
  Tính xác suất có điều kiện P(X=x|Y=y) cho mỗi feature
  
  Công thức: P(X=x|Y=y) = số mẫu có (X=x và Y=y) / số mẫu có Y=y
  
  Ví dụ: P(Outlook='Sunny'|PlayTennis='Yes')
         = số lần (Outlook='Sunny' và PlayTennis='Yes') / số lần PlayTennis='Yes'
  
  Args:
    train_data: numpy array dữ liệu huấn luyện
    
  Returns:
    - conditional_probability: list chứa ma trận xác suất của từng feature
      + Mỗi ma trận có shape (2, n_values): 2 hàng (no/yes), n_values cột (các giá trị của feature)
    - list_x_name: list chứa các giá trị unique của từng feature
  """
  y_unique = ['no', 'yes']
  conditional_probability = []
  list_x_name = []
  
  # Duyệt qua từng feature (bỏ cột cuối là label)
  for i in range(train_data.shape[1] - 1):
    # Lấy tất cả giá trị unique của feature thứ i
    # Ví dụ: Outlook có 3 giá trị [Overcast, Rain, Sunny]
    x_unique = np.unique(train_data[:, i])
    list_x_name.append(x_unique)
    
    # Tạo ma trận lưu xác suất có điều kiện
    # Hàng: lớp (no=0, yes=1), Cột: giá trị của feature
    x_conditional_probability = np.zeros((len(y_unique), len(x_unique)))
    
    # Tính P(X=x|Y=y) cho từng tổ hợp
    for j in range(len(y_unique)):        # Duyệt qua từng lớp (no, yes)
      for k in range(len(x_unique)):      # Duyệt qua từng giá trị của feature
        # Đếm số mẫu có: feature=x_unique[k] VÀ label=y_unique[j]
        count = np.sum((train_data[:, i] == x_unique[k]) & (train_data[:, -1] == y_unique[j]))
        
        # Đếm tổng số mẫu có label=y_unique[j]
        total = np.sum(train_data[:, -1] == y_unique[j])
        
        # Xác suất = count / total
        x_conditional_probability[j, k] = count / total
    
    conditional_probability.append(x_conditional_probability)
  
  return conditional_probability, list_x_name

# Test
train_data = create_train_data()
conditional_probability, list_x_name = compute_conditional_probability(train_data)

print("Các giá trị của từng feature:")
print(f"x1 (Outlook) = {list_x_name[0]}")
print(f"x2 (Temperature) = {list_x_name[1]}")
print(f"x3 (Humidity) = {list_x_name[2]}")
print(f"x4 (Wind) = {list_x_name[3]}")

Các giá trị của từng feature:
x1 (Outlook) = ['Overcast' 'Rain' 'Sunny']
x2 (Temperature) = ['Cool' 'Hot' 'Mild']
x3 (Humidity) = ['High' 'Normal']
x4 (Wind) = ['Strong' 'Weak']


In [27]:
# ============================================
# BƯỚC 4: KIỂM TRA INDEX CỦA CÁC GIÁ TRỊ
# ============================================

# Hàm trợ giúp: tìm index của một giá trị trong array
def getIndexFromValue(featureName, listFeatures):
  """
  Tìm vị trí (index) của một giá trị trong array
  
  Args:
    featureName: giá trị cần tìm (vd: "Sunny")
    listFeatures: array chứa các giá trị (vd: ['Overcast', 'Rain', 'Sunny'])
    
  Returns:
    index của giá trị trong array
  """
  return np.where(listFeatures == featureName)[0][0]

# Test: Tìm index của các giá trị Outlook
outlook = list_x_name[0]
i1 = getIndexFromValue("Overcast", outlook)
i2 = getIndexFromValue("Rain", outlook)
i3 = getIndexFromValue("Sunny", outlook)
print(f"Index của Overcast, Rain, Sunny: {i1}, {i2}, {i3}")

Index của Overcast, Rain, Sunny: 0, 1, 2


In [28]:
# ============================================
# BƯỚC 6: HÀM TRAINING TỔNG HỢP
# ============================================

def train_naive_bayes(train_data):
  """
  Huấn luyện mô hình Naive Bayes
  
  Kết hợp 2 bước:
  1. Tính xác suất tiên nghiệm P(Y)
  2. Tính xác suất có điều kiện P(X|Y)
  
  Returns:
    - prior_probability: xác suất tiên nghiệm
    - conditional_probability: xác suất có điều kiện
    - list_x_name: danh sách giá trị của các feature
  """
  prior_probability = compute_prior_probablity(train_data)
  conditional_probability, list_x_name = compute_conditional_probability(train_data)
  return prior_probability, conditional_probability, list_x_name

# Huấn luyện mô hình
data = create_train_data()
prior_probability, conditional_probability, list_x_name = train_naive_bayes(data)
print("✓ Đã huấn luyện xong mô hình Naive Bayes!")

✓ Đã huấn luyện xong mô hình Naive Bayes!


In [29]:
#This function is used to return the index of the feature name
def getIndexFromValue(featureName, listFeatures):
  return np.where(listFeatures == featureName)[0][0]

In [30]:
# ============================================
# BƯỚC 5: TÍNH VÍ DỤ XÁC SUẤT CÓ ĐIỀU KIỆN
# ============================================

# Ví dụ: Tính P(Outlook='Sunny' | PlayTennis='Yes')
x1 = getIndexFromValue("Sunny", list_x_name[0])
prob = conditional_probability[0][1, x1]  # [0]=Outlook, [1]=yes, [x1]=Sunny
print(f"P(Outlook='Sunny' | PlayTennis='Yes') = {np.round(prob, 2)}")

# Ví dụ: Tính P(Outlook='Sunny' | PlayTennis='No')
prob_no = conditional_probability[0][0, x1]  # [0]=Outlook, [0]=no, [x1]=Sunny
print(f"P(Outlook='Sunny' | PlayTennis='No') = {np.round(prob_no, 2)}")

P(Outlook='Sunny' | PlayTennis='Yes') = 0.17
P(Outlook='Sunny' | PlayTennis='No') = 0.75


In [31]:
# ============================================
# BƯỚC 7: HÀM DỰ ĐOÁN
# ============================================

def prediction_play_tennis(X, list_x_name, prior_probability, conditional_probability):
  """
  Dự đoán nhãn cho một mẫu mới bằng Naive Bayes
  
  Công thức: P(Y|X) = P(Y) × P(X₁|Y) × P(X₂|Y) × P(X₃|Y) × P(X₄|Y)
  
  Các bước:
  1. Tìm index của từng giá trị feature trong X
  2. Tính P(Y='no'|X) và P(Y='yes'|X)
  3. Chọn lớp có xác suất cao hơn
  
  Args:
    X: mẫu cần dự đoán [Outlook, Temperature, Humidity, Wind]
    list_x_name: danh sách giá trị của các feature
    prior_probability: xác suất tiên nghiệm P(Y)
    conditional_probability: xác suất có điều kiện P(X|Y)
    
  Returns:
    0 nếu dự đoán 'no', 1 nếu dự đoán 'yes'
  """
  # Bước 1: Lấy index của từng feature
  x1 = getIndexFromValue(X[0], list_x_name[0])  # Outlook
  x2 = getIndexFromValue(X[1], list_x_name[1])  # Temperature
  x3 = getIndexFromValue(X[2], list_x_name[2])  # Humidity
  x4 = getIndexFromValue(X[3], list_x_name[3])  # Wind
  
  # Bước 2: Tính P(Y='no'|X)
  # P(Y='no'|X) = P(Y='no') × P(Outlook|Y='no') × P(Temp|Y='no') × P(Humid|Y='no') × P(Wind|Y='no')
  p0 = prior_probability[0]                      # P(Y='no')
  p0 *= conditional_probability[0][0, x1]        # × P(Outlook|Y='no')
  p0 *= conditional_probability[1][0, x2]        # × P(Temperature|Y='no')
  p0 *= conditional_probability[2][0, x3]        # × P(Humidity|Y='no')
  p0 *= conditional_probability[3][0, x4]        # × P(Wind|Y='no')
  
  # Bước 3: Tính P(Y='yes'|X)
  # P(Y='yes'|X) = P(Y='yes') × P(Outlook|Y='yes') × ... (tương tự)
  p1 = prior_probability[1]                      # P(Y='yes')
  p1 *= conditional_probability[0][1, x1]        # × P(Outlook|Y='yes')
  p1 *= conditional_probability[1][1, x2]        # × P(Temperature|Y='yes')
  p1 *= conditional_probability[2][1, x3]        # × P(Humidity|Y='yes')
  p1 *= conditional_probability[3][1, x4]        # × P(Wind|Y='yes')
  
  print(f"P(Y='no'|X) = {p0:.6f}")
  print(f"P(Y='yes'|X) = {p1:.6f}")
  
  # Bước 4: Chọn lớp có xác suất cao hơn
  if p0 > p1:
    return 0  # Dự đoán 'no'
  else:
    return 1  # Dự đoán 'yes'

In [32]:
# ============================================
# BƯỚC 8: KIỂM TRA DỰ ĐOÁN
# ============================================

# Mẫu cần dự đoán: Sunny, Cool, High, Strong
X = ['Sunny', 'Cool', 'High', 'Strong']
print(f"Dự đoán cho: {X}\n")

# Huấn luyện và dự đoán
data = create_train_data()
prior_probability, conditional_probability, list_x_name = train_naive_bayes(data)
pred = prediction_play_tennis(X, list_x_name, prior_probability, conditional_probability)

# Kết quả
print(f"\n{'='*40}")
if pred:
  print("🎾 Dự đoán: Ad should go! (Nên chơi tennis)")
else:
  print("❌ Dự đoán: Ad should not go! (Không nên chơi tennis)")
print('='*40)

Dự đoán cho: ['Sunny', 'Cool', 'High', 'Strong']

P(Y='no'|X) = 0.028125
P(Y='yes'|X) = 0.002778

❌ Dự đoán: Ad should not go! (Không nên chơi tennis)


#### Question 5


## Gaussian Naive Bayes - Iris Classification

**Khác biệt với Question 4:**
- Question 4: Dữ liệu **rời rạc** (categorical) → dùng tần suất
- Question 5: Dữ liệu **liên tục** (continuous) → dùng **phân phối Gaussian**

**Gaussian Naive Bayes:**
```
P(X=x|Y=y) = (1 / √(2πσ²)) × exp(-(x-μ)² / (2σ²))
```

Trong đó:
- μ (mean): trung bình của feature khi thuộc lớp y
- σ² (variance): phương sai của feature khi thuộc lớp y

**Iris Dataset:**
- 4 features liên tục: sepal length, sepal width, petal length, petal width
- 3 lớp: Iris-setosa, Iris-versicolor, Iris-virginica

### Iris classification

In [ ]:

from sklearn import datasets
import numpy as np

def create_train_data_iris():
  data = np.loadtxt("iris.data.txt", delimiter=",", dtype=str)
  return data



In [ ]:
# ============================================
# BƯỚC 1: TÍNH XÁC SUẤT TIÊN NGHIỆM P(Y)
# ============================================

def compute_prior_probablity_iris(train_data):
  """
  Tính P(Y) cho mỗi loại hoa Iris
  
  Returns:
    numpy array [P(Setosa), P(Versicolor), P(Virginica)]
  """
  y_unique = np.unique(train_data[:, 4])  # Lấy các lớp unique
  prior_probability = np.zeros(len(y_unique))
  
  # Tính tần suất xuất hiện của mỗi lớp
  for i in range(len(y_unique)):
    count = np.sum(train_data[:, 4] == y_unique[i])
    prior_probability[i] = count / len(train_data)
  
  return prior_probability

# Test
train_data_iris = create_train_data_iris()
prior_prob_iris = compute_prior_probablity_iris(train_data_iris)
y_unique = np.unique(train_data_iris[:, 4])
for i, class_name in enumerate(y_unique):
  print(f"P({class_name}) = {prior_prob_iris[i]:.3f}")

In [ ]:
# ============================================
# BƯỚC 2: TÍNH MEAN VÀ VARIANCE CHO GAUSSIAN
# ============================================

def compute_conditional_probability_iris(train_data):
  """
  Tính mean (μ) và standard deviation (σ) cho mỗi feature theo từng lớp
  
  Thay vì tính P(X|Y) trực tiếp, ta lưu μ và σ để dùng trong hàm Gaussian
  
  Returns:
    List chứa ma trận (3, 2) cho mỗi feature:
    - 3 hàng: 3 lớp hoa (Setosa, Versicolor, Virginica)
    - 2 cột: [mean, std] của feature đó
  """
  y_unique = np.unique(train_data[:, 4])  # 3 lớp hoa
  conditional_probability = []
  
  # Duyệt qua 4 features (bỏ cột label)
  for i in range(train_data.shape[1] - 1):
    # Ma trận lưu [mean, std] cho từng lớp
    x_conditional_probability = np.zeros((len(y_unique), 2))
    
    # Tính mean và std cho từng lớp
    for j in range(len(y_unique)):
      # Lấy tất cả giá trị của feature i khi thuộc lớp j
      feature_values = train_data[train_data[:, 4] == y_unique[j], i].astype(float)
      
      # Tính mean (trung bình) và std (độ lệch chuẩn)
      x_conditional_probability[j, 0] = np.mean(feature_values)  # μ
      x_conditional_probability[j, 1] = np.std(feature_values)   # σ
    
    conditional_probability.append(x_conditional_probability)
  
  return conditional_probability

# Test
cond_prob_iris = compute_conditional_probability_iris(train_data_iris)
print("Feature 0 (Sepal Length) - Mean và Std cho 3 lớp:")
print(cond_prob_iris[0])

In [ ]:
# ============================================
# BƯỚC 3: HÀM PHÂN PHỐI GAUSSIAN
# ============================================

import math

def gauss(x, mean, sigma):
  """
  Tính xác suất từ phân phối Gaussian (phân phối chuẩn)
  
  Công thức: P(x) = (1/√(2πσ²)) × exp(-(x-μ)²/(2σ²))
  
  Args:
    x: giá trị cần tính xác suất
    mean: trung bình (μ)
    sigma: độ lệch chuẩn (σ)
    
  Returns:
    xác suất của x theo phân phối Gaussian
  """
  # Tránh chia cho 0
  if sigma == 0:
    sigma = 1e-6
  
  # Tính từng thành phần
  coefficient = 1.0 / (math.sqrt(2 * math.pi) * sigma)
  exponent = math.exp(-((x - mean) ** 2) / (2 * sigma ** 2))
  
  result = coefficient * exponent
  return result

# Test
x_test = 5.0
mean_test = 5.8
sigma_test = 0.8
prob = gauss(x_test, mean_test, sigma_test)
print(f"P(x={x_test} | μ={mean_test}, σ={sigma_test}) = {prob:.6f}")

In [ ]:
# ============================================
# BƯỚC 4: TRAINING GAUSSIAN NAIVE BAYES
# ============================================

def train_gaussian_naive_bayes(train_data):
  """
  Huấn luyện mô hình Gaussian Naive Bayes cho Iris
  
  Kết hợp:
  1. Tính prior probability P(Y)
  2. Tính mean và std cho mỗi feature theo từng lớp
  
  Returns:
    - prior_probability: P(Y)
    - conditional_probability: list chứa [mean, std] cho mỗi feature
  """
  prior_probability = compute_prior_probablity_iris(train_data)
  conditional_probability = compute_conditional_probability_iris(train_data)
  return prior_probability, conditional_probability

# Test
data_iris = create_train_data_iris()
prior_prob, cond_prob = train_gaussian_naive_bayes(data_iris)
print("✓ Đã huấn luyện xong Gaussian Naive Bayes cho Iris!")

In [ ]:
# ============================================
# BƯỚC 5: DỰ ĐOÁN BẰNG GAUSSIAN NAIVE BAYES
# ============================================

def prediction_iris(X, prior_probability, conditional_probability):
  """
  Dự đoán loại hoa Iris bằng Gaussian Naive Bayes
  
  Công thức: P(Y|X) = P(Y) × Π[gauss(xᵢ, μᵢ, σᵢ)]
  
  Các bước:
  1. Tính P(Y=Setosa|X) = P(Y=Setosa) × gauss(x₁) × gauss(x₂) × gauss(x₃) × gauss(x₄)
  2. Tính P(Y=Versicolor|X) tương tự
  3. Tính P(Y=Virginica|X) tương tự
  4. Chọn lớp có xác suất cao nhất
  
  Args:
    X: mẫu cần dự đoán [sepal_length, sepal_width, petal_length, petal_width]
    prior_probability: xác suất tiên nghiệm P(Y)
    conditional_probability: list chứa [mean, std] cho mỗi feature
    
  Returns:
    index của lớp được dự đoán (0, 1, hoặc 2)
  """
  # Khởi tạo xác suất cho 3 lớp
  p0 = prior_probability[0]  # P(Y=Setosa)
  p1 = prior_probability[1]  # P(Y=Versicolor)
  p2 = prior_probability[2]  # P(Y=Virginica)
  
  # Nhân với P(X|Y) từ Gaussian cho từng feature
  for i in range(len(X)):  # Duyệt qua 4 features
    # P(feature_i | Y=Setosa)
    mean_0 = conditional_probability[i][0, 0]
    std_0 = conditional_probability[i][0, 1]
    p0 *= gauss(X[i], mean_0, std_0)
    
    # P(feature_i | Y=Versicolor)
    mean_1 = conditional_probability[i][1, 0]
    std_1 = conditional_probability[i][1, 1]
    p1 *= gauss(X[i], mean_1, std_1)
    
    # P(feature_i | Y=Virginica)
    mean_2 = conditional_probability[i][2, 0]
    std_2 = conditional_probability[i][2, 1]
    p2 *= gauss(X[i], mean_2, std_2)
  
  # Chọn lớp có xác suất cao nhất
  list_p = [p0, p1, p2]
  return list_p.index(np.max(list_p))

In [ ]:
#Example 1 #########################
X = [6.3 , 3.3, 6.0,  2.5]
train_data = create_train_data_iris()
y_unique = np.unique(train_data[:,4])
prior_probability, conditional_probability = train_gaussian_naive_bayes(train_data)
pred = y_unique[prediction_iris(X, prior_probability, conditional_probability)]
assert pred == "Iris-virginica"

In [ ]:
#Example 2 #########################
X = [5.0,2.0,3.5,1.0]
train_data = create_train_data_iris()
y_unique = np.unique(train_data[:,4])
prior_probability, conditional_probability = train_gaussian_naive_bayes(train_data)
pred = y_unique[prediction_iris(X, prior_probability, conditional_probability)]
assert pred == "Iris-versicolor"

In [ ]:
#Example 3 #########################
# X =[sepal length, sepal width, petal length, petal width]
X = [4.9,3.1,1.5,0.1]
train_data = create_train_data_iris()
y_unique = np.unique(train_data[:,4])
prior_probability, conditional_probability = train_gaussian_naive_bayes(train_data)
pred = y_unique[prediction_iris(X, prior_probability, conditional_probability)]
assert pred == "Iris-setosa"